In [1]:
import os
import json
import pandas as pd
import numpy as np

current_directory = os.getcwd()
path_components = current_directory.split(os.path.sep)
results_dir = os.path.sep.join(path_components[:-3])


In [2]:
def find_largest_iteration(folder_path):
    subdirectories = [d for d in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, d))]
    iteration_numbers = [int(d.split('_')[1]) for d in subdirectories if d.startswith('iter_')]
    if not iteration_numbers:
        return None
    largest_iteration_folder = os.path.join(folder_path, f'iter_{max(iteration_numbers)}')
    return largest_iteration_folder

def find_loss_csv(folder_path):
    largest_iteration_folder = find_largest_iteration(os.path.join(folder_path,'iterations'))
    if largest_iteration_folder:
        loss_csv_path = os.path.join(largest_iteration_folder, 'loss.csv')
        if os.path.isfile(loss_csv_path):
            return loss_csv_path
    return None

def get_last_iteration_losses(loss_csv_path):
    df = pd.read_csv(loss_csv_path)
    last_row = df.iloc[-1]
    losses = last_row[list(df.columns)]
    return losses

def read_results_json(folder_path):
    results_json_path = os.path.join(folder_path, 'results_values.json')
    if os.path.isfile(results_json_path):
        with open(results_json_path, 'r') as json_file:
            results_data = json.load(json_file)
        return results_data
    else:
        return None

def get_results_by_sim(folder_path):
    loss_csv_path = find_loss_csv(folder_path)
    if loss_csv_path:
        results = get_last_iteration_losses(loss_csv_path)
        results_data = read_results_json(folder_path)
        if results_data:
            for name in results_data:
                results[name] = float(results_data[name])
            return results
    return None

In [3]:
molecule = 'born_ion'
results_dir = os.path.join(results_dir,'.results',molecule)
print(results_dir)

/home/martinachondo/XPBE/.results/born_ion


In [6]:
folders = ['.architecture', '.scheme_1']
data = dict()
for folder in folders:
    folder_path = os.path.join(results_dir,folder) if folder!= '' else results_dir
    sims = os.listdir(folder_path)
    for sim in sims:
        sim_path = os.path.join(folder_path,sim)
        result = get_results_by_sim(sim_path)
        data[sim] = result

data_filtered = {sim: result for sim, result in data.items() if result is not None}
df = pd.DataFrame.from_dict(data_filtered, orient='index')

filter_list = ["Gsolv_value",
                "Loss_XPINN",
                "Loss_NN1",
                "Loss_NN2",
                "Loss_Val_NN1",
                "Loss_Val_NN2",
                "Loss_continuity_u",
                "Loss_continuity_du",
                "Loss_Residual_R1",
                "Loss_Residual_R2",
                "Loss_Boundary_D2",
                "Loss_Data_K2",
                "L2_analytic"]
df_filtered = df.filter(filter_list)

excel_file_path = 'results_temp.xlsx'
with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='DF_complete', index=True)
    df_filtered.to_excel(writer, sheet_name='DF_Filtered', index=True)

In [7]:
df_filtered

,Gsolv_value,Loss_XPINN,Loss_NN1,Loss_NN2,Loss_Val_NN1,Loss_Val_NN2,Loss_continuity_u,Loss_continuity_du,Loss_Residual_R1,Loss_Residual_R2,Loss_Boundary_D2,Loss_Data_K2,L2_analytic
B13_H4_N200,-165.255,6.527000e-03,5.018000e-03,0.005018,4.926000e-03,0.004926,7.650000e-08,5.018000e-03,1.259000e-14,6.791000e-08,2.417000e-08,4.154000e-08,42.430
B6_FO_H2_N200_E_FA,-165.388,1.180000e-02,1.532000e-03,0.001040,1.662000e-03,0.000930,8.895000e-07,1.021000e-03,5.096000e-04,1.789000e-05,1.972000e-07,2.538000e-07,52.290
B7_FO_H6_N200_E_FA,-165.311,1.049000e-03,7.149000e-05,0.000020,2.684000e-04,0.000134,3.433000e-08,9.089000e-07,7.054000e-05,1.878000e-05,1.761000e-07,8.551000e-08,47.660
B11_FO_H4_N200_E,-164.610,2.805000e-03,2.411000e-04,0.000065,5.838000e-04,0.000307,1.033000e-07,2.031000e-05,2.207000e-04,4.419000e-05,4.364000e-08,1.726000e-08,62.280
B8_FO_H4_N100_E_FA,-165.336,2.075000e-03,4.481000e-04,0.000008,4.037000e-04,0.000008,3.115000e-08,6.624000e-06,4.414000e-04,1.539000e-06,3.338000e-09,5.685000e-09,47.630
B12_FO_H4_N200,-166.418,6.052000e-03,5.808000e-04,0.000920,8.260000e-04,0.001288,1.376000e-08,5.751000e-04,5.645000e-06,3.446000e-04,5.104000e-09,4.875000e-09,21.750
B4_FO_H4_N200_E_FA,-164.719,1.927000e-03,4.400000e-04,0.000030,5.848000e-04,0.000037,1.203000e-07,2.632000e-05,4.135000e-04,3.643000e-06,1.601000e-08,1.155000e-08,56.620
B18_FO_H4_N200_E_FA_WF_ResNet,-162.708,2.343000e+10,6.329000e+08,775500.000000,6.987000e+07,89050.000000,1.598000e-05,5.782000e+00,6.329000e+08,7.755000e+05,5.532000e-06,9.167000e-07,102.800
B17_FO_H2_N200_E_FA_WF_ModMLP,-165.414,1.176000e-02,2.184000e-04,0.000177,4.002000e-04,0.000335,6.856000e-07,1.681000e-04,4.965000e-05,8.176000e-06,4.316000e-08,8.289000e-08,48.740
B15_FO_H2_N200_E_FA_ModMLP,-166.399,1.908000e-02,1.376000e-03,0.000937,1.476000e-03,0.000999,1.422000e-07,9.091000e-04,4.664000e-04,2.764000e-05,6.546000e-08,4.982000e-08,57.830
